In [1]:
# using linear support vector machine to predict speaker from text!
# the internet says it is one of the best algorithms for text classification, so that's why i'll be using it 
# also, maybe only predicting two labels was making it difficult for the naive bayes. trying speakers here because that's a lot more labels (and it is more fun!)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn 
print(sklearn.__version__)


0.24.1


In [3]:
df = pd.read_csv('The-Office-Lines-V4.csv')
df= df.drop('Unnamed: 6', 1)

In [4]:
def gender(name):
    if name == 'Michael' or name == 'Jim' or name == 'Dwight' or name == 'Ryan' or name == 'Stanley' or name == 'Kevin' or name == 'Oscar' or name == 'Toby' or name == 'Creed' or name == 'Andy' or name == 'Roy' or name == 'Darryl' or name == 'Gabe' or name == 'Robert California' or name == 'Robert' or name == 'Clark' or name == 'Pete':
        return 0
    elif name == 'Pam' or name == 'Phyllis' or name == 'Angela' or name == 'Kelly' or name == 'Meredith' or name == 'Jan' or name == 'Erin' or name == 'Nellie':
        return 1

In [5]:
df["gender"] = df["speaker"].map(gender)

In [6]:
df = df.dropna()

In [7]:
df['gender'] = df['gender'].astype(int)

In [8]:
df['number_of_words'] = df['line'].str.split().str.len()

In [9]:
df['speaker_id'] = pd.factorize(df.speaker)[0]

In [10]:
df.sample()

,season,episode,title,scene,speaker,line,gender,number_of_words,speaker_id
11616,3,18,Cocktails,2012,Jan,Shut up.,1,2,4


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [12]:
X = df['line']
y = df['speaker']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=.33)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31934,), (31934,), (15730,), (15730,))

In [15]:
vect = CountVectorizer()
tfidf = TfidfTransformer()
sgd = SGDClassifier()

In [16]:
pipe = make_pipeline(vect, tfidf, sgd)

In [17]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('tfidftransformer', TfidfTransformer()),
                ('sgdclassifier', SGDClassifier())])

In [18]:
X_train

28463                                   Okay.  You owe me.
10883                                               Sally.
51277                        No. She needs her rest again.
4371      What do you do with a drunken sailor? What do...
27832                                       And oh, Jim...
                               ...                        
12558                                                  OK.
51152                   All right. I think that went well.
43751                 There is one problem with this plan.
910                               It's just office pranks.
17535                 Hey Pam, I really like your glasses.
Name: line, Length: 31934, dtype: object

In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [20]:
my_tags = ['Michael', 'Jim', 'Pam', 'Dwight', 'Jan', 'Phyllis', 'Stanley',
       'Oscar', 'Angela', 'Kevin', 'Ryan', 'Roy', 'Toby', 'Kelly',
       'Meredith', 'Darryl', 'Creed', 'Andy', 'Pete', 'Erin', 'Gabe',
       'Clark', 'Robert', 'Robert California', 'Nellie']

In [21]:
y_pred = pipe.predict(X_test)

In [22]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.23808010171646535


C:\Users\maret\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

          Michael       0.16      0.17      0.17      1223
              Jim       0.17      0.08      0.11       496
              Pam       0.01      0.01      0.01        97
           Dwight       0.08      0.04      0.06       135
              Jan       0.18      0.12      0.15       361
          Phyllis       0.26      0.33      0.29      2232
          Stanley       0.11      0.07      0.09       469
            Oscar       0.07      0.06      0.07       139
           Angela       0.10      0.05      0.07       268
            Kevin       0.22      0.21      0.21      2037
             Ryan       0.09      0.05      0.07       284
              Roy       0.09      0.07      0.08       530
             Toby       0.06      0.04      0.05       184
            Kelly       0.33      0.52      0.40      3545
         Meredith       0.16      0.06      0.09       162
           Darryl       0.10      0.05      0.06       

C:\Users\maret\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maret\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
# trying to do it like in class, but SVC() causes problems (in class we were taking a numeric value and predicting a binary value; here, it is text to numeric(speaker_id))
# it tries to convert the input lines to floats, which isnt possible. 

#using tfidftransformer should fix this; transform text into numeric data

from sklearn.pipeline import make_pipeline

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.svm import LinearSVC, SVC 
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer


# imputer = KNNImputer()
vect = CountVectorizer()
scaler = MaxAbsScaler()
tfidf = TfidfTransformer()
# scaler = MinMaxScaler()
svm1 = LinearSVC()
svm2 = SVC()


In [24]:
pipe1 = make_pipeline(vect, scaler, tfidf, svm1)

In [25]:
pipe2 = make_pipeline(vect, scaler, tfidf, svm2)

In [26]:
svm1.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [27]:
svm2.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [28]:
X = df['line']
y = df['speaker_id']

In [29]:
scores = cross_val_score(pipe1, X, y, cv=10, scoring='f1_macro') # only measures the harmonic mean of the positive class
print(f"Mean {scores.mean()}, Std {scores.std()}")


Mean 0.0690114604166341, Std 0.006894584452084698


In [30]:
# dont have to run this (takes too long idk why) bc im checking the best kernel in the next cell
# scores = cross_val_score(pipe2, X, y, cv=10, scoring='f1_macro') # only measures the harmonic mean of the positive class (here: survived)
# print(f"Mean {scores.mean()}, Std {scores.std()}")


# poly and rbf kernels take very long to run (actually they dont run at all) so were sticking to linear kernel

In [31]:
# kernels = ['linear', 'poly'] # poly for polynomial # defining the diff kernels here, you want to see if there is any difference between these three
# accuracies= [] 

# for k in kernels:
#     svm = SVC(kernel=k) # initializing your support vector machine
#     pipe3 = make_pipeline(vect, tfidf, svm)
#     scores = cross_val_score(pipe3, X, y, cv=10, scoring='f1_macro') # doing crossvalidation to see if the model is robust
#     accuracies.append(scores.mean())

# plt.plot(kernels, accuracies)
# plt.show()


# # this wont run!! something with rbf and poly kernels, because linear does work on its own

In [32]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('maxabsscaler', MaxAbsScaler()),
                ('tfidftransformer', TfidfTransformer()),
                ('linearsvc', LinearSVC())])

In [33]:
y_test_pred = pipe1.predict(X_test)

In [34]:
print(classification_report(y_test, y_test_pred,target_names=my_tags))

                   precision    recall  f1-score   support

          Michael       0.20      0.13      0.16      1223
              Jim       0.17      0.08      0.11       496
              Pam       0.04      0.01      0.02        97
           Dwight       0.15      0.04      0.07       135
              Jan       0.17      0.10      0.12       361
          Phyllis       0.26      0.27      0.27      2232
          Stanley       0.14      0.07      0.10       469
            Oscar       0.11      0.04      0.05       139
           Angela       0.08      0.03      0.04       268
            Kevin       0.21      0.27      0.24      2037
             Ryan       0.15      0.05      0.08       284
              Roy       0.13      0.08      0.09       530
             Toby       0.09      0.04      0.05       184
            Kelly       0.29      0.53      0.38      3545
         Meredith       0.13      0.06      0.08       162
           Darryl       0.11      0.06      0.08       

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
param_grid = {'linearsvc__C': [0.1, 1, 10, 100, 1000],
               'countvectorizer__ngram_range': [(1,1),(1,2),(2,2)], 
               'tfidftransformer__use_idf': (True, False),
               'tfidftransformer__norm': ('l1', 'l2')
               }

In [37]:
gs = GridSearchCV(pipe1, param_grid, cv=3, refit = True, verbose = 3)

In [38]:
# param_grid2 = {'C': [0.1, 1, 10, 100, 1000],
#              'verbose': [5],
#              'loss': ('hinge', 'squared_hinge'),
#              'multi_class': ('ovr', 'crammer_singer')}

In [39]:
# gs2 = GridSearchCV(svm1, param_grid2, cv=3, scoring='accuracy', n_jobs=-1, refit = True, verbose = 3)

In [40]:
# experimental_pipe = make_pipeline(vect, tfidf, gs2)

In [41]:
# experimental_pipe.fit(X_train, y_train)

In [42]:
# this takes hours and doesnt run; maybe gridsearch enters infinite loop; try looking for best parameters manually
# (kernel, C, gamma)
# gs.fit(X_train,y_train)

In [43]:
# trying rbf kernel manually, without gridsearch
# vect = CountVectorizer()
# scaler = MaxAbsScaler()
# tfidf = TfidfTransformer()
# svm2 = SVC()

# pipe2 = make_pipeline(vect, scaler, tfidf, svm2)

In [44]:
# also doesnt run
# pipe2.fit(X_train, y_train)

In [45]:
# y_test_pred = pipe1.predict(X_test)

In [46]:
# print(classification_report(y_test, y_test_pred,target_names=my_tags))

In [ ]:
# CONCLUSION : gridsearch causes a lot of trouble, and wont let anything run
# trying other kernels manually also doesnt work
# however, i tried running all this in a google colab notebook, and it returned the highest score for the linear kernel
# so we will stick to the linear kernel!